In [2]:
import requests
import json
import csv
from datetime import datetime
import pytz

# URL for the API endpoint
url = "https://gis.mtdeq.us/hosting/rest/services/Hosted/montana_aqmon_stations_dataportal/FeatureServer/0/query"

# Parameters for the API request
params = {
    'f': 'json',
    'where': "sitename='Butte' AND parameter='PM25'",
    'outFields': '*',
    'returnGeometry': 'false',
    'resultRecordCount': 1000  # The number of records to retrieve per request
}

# Open a file for writing
with open('output.csv', 'w', newline='') as csvfile:
    writer = None

    # Initialize the offset
    params['resultOffset'] = 0

    while True:
        # Send the API request
        response = requests.get(url, params=params)
        
        # Check if the request was successful
        response.raise_for_status()

        # Load the data from the response
        data = json.loads(response.text)

        # If no features are returned, we're done
        if not data['features']:
            break

        # Extract the features from the data
        features = data['features']

        # If this is the first request, initialize the writer
        if writer is None:
            # Extract the field names from the first feature (assuming all features have the same fields)
            field_names = features[0]['attributes'].keys()
            writer = csv.DictWriter(csvfile, fieldnames=field_names)
            # Write the header
            writer.writeheader()

        # Write the data
        for feature in features:
            # Convert the 'datetime' and 'date' fields from Unix time to datetime objects
            for field in ['datetime', 'date']:
                unix_time = feature['attributes'][field]
                utc_time = datetime.fromtimestamp(unix_time / 1000, tz=pytz.UTC)
                
                # Convert the UTC time to the 'America/Denver' time zone
                mt_time = utc_time.astimezone(pytz.timezone('America/Denver'))

                # Replace the field with the formatted time
                feature['attributes'][field] = mt_time.isoformat()

            writer.writerow(feature['attributes'])

        # Update the offset
        params['resultOffset'] += params['resultRecordCount']
